<h1>PEST-python: Python Interfaces for PEST </h1>

<hr>
DISTRIBUTION STATEMENT A. Approved for public release. Distribution is unlimited.

This material is based upon work supported by the Under Secretary of Defense for
Research and Engineering under Air Force Contract No. FA8702-15-D-0001. Any opinions,
findings, conclusions or recommendations expressed in this material are those of the
author(s) and do not necessarily reflect the views of the Under Secretary of Defense
for Research and Engineering.

© 2023 Massachusetts Institute of Technology.

The software/firmware is provided to you on an As-Is basis

Delivered to the U.S. Government with Unlimited Rights, as defined in DFARS Part
252.227-7013 or 7014 (Feb 2014). Notwithstanding any copyright notice, U.S. Government
rights in this work are defined by DFARS 252.227-7013 or DFARS 252.227-7014 as detailed
above. Use of this work other than as specifically authorized by the U.S. Government
may violate any copyrights that exist in this work.
<hr><hr>

<b>PEST-python</b> provides interface methods for PEST: a library of Julia-based electronic structure codes for `pyLIQTR`.  These wrappers can be used to (i) run the command-line version of PEST from within Python scripts and (ii) read PEST output into a Python workflow.  The discussion in this notebook documents these methods, while a general discussion of PEST is given in `pest-intro.ipynb`.

<h3><u>Loading PEST-python:</u></h3>

In [17]:
import  numpy        as  np
import  pest_python  as  pp

<h3><u>Reading PEST Output</u></h3>

PEST output is manipulated using the `HamiltonianCoefficients` class.  We can load a PEST Hamiltonian into the associated container using the `open_grid()` and `open_hamiltonian()` methods.  The example below uses a precomputed reference set for cyclic ozone ($\text{O}_3$) encapsulated by fullerene cage ($\text{C}_{60}$):

In [19]:
H = pp.HamiltonianCoefficients()

H.open_grid('../../data/hamiltonians/cyclic_ozone/C60-o3-opt-pad_7.5_Ecut_10.0_scale_1.0.grid.hdf5')
H.open_hamiltonian('../../data/hamiltonians/cyclic_ozone/C60-o3-opt-pad_7.5_Ecut_10.0_scale_1.0.ham.hdf5')


These data correspond to a DPW Hamiltonian.  To see the number of real-space grid points (basis functions) that define this system along each dimension, we can use the `n_points()`:


In [20]:
n_points = H.n_points()
n_points

array([14, 15, 15])

The total number of basis functions is then:

In [21]:
N = np.prod(n_points)
N

3150

The `HamiltonianCoefficients` object provides a table that relates each grid coordinates to a scalar index:

In [22]:
H.gridpt_to_id([1,1,2])

242

A similar mapping is also provided for the distance vector between indices. For instance, the vector $\vec{d}_{1,2} = \vec{r}_1 - \vec{r}_2$ is given by:

In [23]:
r_1  =  np.array([0,0,1])
r_3  =  np.array([0,0,3])

d_1_3 = r_1 - r_3
d_1_3

array([ 0,  0, -2])

Wich has a distance index given by:

In [24]:
H.distance_dict(d_1_3)

4

The coefficients for a given Hamiltonian term are accessed using the scalar indices from `gridpt_to_id`.  For instance, the kinetic term $T(p-q) \,c^\dagger_p c_q$ coupling grid points $p = r_1$ and $q = r_2$ has a coefficient $T(p-q)$ given by:

In [25]:
idx_1  =  H.gridpt_to_id(r_1)
idx_3  =  H.gridpt_to_id(r_3)

T_1_3 = H.T(idx_1,idx_3)
T_1_3

0.06992250338350335

While the coefficient  $V(p-q)$ from the electron-electron interaction term $V(p-q)\,n_p n_q$ is:

In [26]:
V_1_3 = H.V(idx_1,idx_3)
V_1_3

0.07710189501831369

With a similar access method is provided for the electron-ion coupling at each grid point:

In [27]:
U_1 = H.U(idx_1)
U_3 = H.U(idx_3)

print(U_1)
print(U_3)

9.100495248126224
7.964596330330162


There are similar access method for the Pauli string coefficients (if they have been calculated).  The coefficients of the $X_p \vec{Z} X_q$ / $Y_p \vec{Z} Y_q$ and $Z_p \otimes Z_q$ terms are:

In [28]:
XZX_1_3 = H.c_xzx(idx_1,idx_3)
XZX_1_3

0.034961251691751676

In [29]:
ZZ_1_3 = H.c_zz(idx_1,idx_3)
ZZ_1_3

0.019275473754578422

And the single $Z_p$ contributions are given by:

In [30]:
Z_1 = H.c_z(idx_1)
Z_3 = H.c_z(idx_3)

print(Z_1)
print(Z_3)

-5.243389447608354
-4.675439988710322


The constant offset can also be obtained:

In [31]:
C_I = H.c_i()
C_I

4366.793541808929

<b>TODO:</b> Decide convention for the constant terms.  The Google papers omit part of this offset; we can add this on explicitly, but that might cause confusion (this is largerly irrelevant for now anyway).

If we want to build arrays containing all coefficients for a Hamiltonian, we might use a method like the following:

In [32]:
# c_xzx_arr  =  np.zeros((N,N))
# c_zz_arr   =  np.zeros((N,N))
# c_z_arr    =  np.zeros(N)


# for j in range(N):
#     for k in range(N):
#         c_xzx_arr[j,k]  =  H.c_xzx(j,k)
#         c_zz_arr[j,k]   =  H.c_zz(j,k)

#     c_z_arr[j]  =  H.c_z(j)


However, this is generally unnecessary due to the provided access methods (many coefficients are either zero or symmetry-related).